[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/pixeltable/pixeltable/blob/master/docs/tutorials/openai-demo.ipynb)&nbsp;&nbsp;
<a target="_blank" href="https://colab.research.google.com/github/pixeltable/pixeltable/blob/master/docs/tutorials/openai-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Using the OpenAI API with Pixeltable

PixelTable helps users unify data and computation into a table interface. This notebook shows how we can use the OpenAI API via pixeltable.
We will keep all artifacts within the `demos` directory.

In [ ]:
%pip install -q pixeltable openai

In [1]:
import pixeltable as pxt
pxt.create_dir('demos', ignore_errors=True)

Connected to Pixeltable database at: postgresql://postgres:@/pixeltable?host=/Users/asiegel/.pixeltable/pgdata


# Creating a table with inputs

In [2]:
path = 'demos.chat_completions_demo'
pxt.drop_table(path, ignore_errors=True)
t = pxt.create_table(path, {'id': pxt.IntType(), 'input': pxt.StringType()})

Created table `chat_completions_demo`.


In [3]:
# text from https://en.wikipedia.org/wiki/Global_financial_crisis_in_September_2008

wikipedia_text = '''On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.
The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to "settle securities transactions with customers of Lehman and its clearance parties" according to a statement made in a New York City Bankruptcy court filing.[18]
The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.
On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.
The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]'''

sample_inputs = wikipedia_text.split('\n')

In [4]:
# example row inserted, persisted to table.
t.insert(id=0, input=sample_inputs[0])
t.show()

Inserting rows into `chat_completions_demo`: 1 rows [00:00, 148.26 rows/s]
Inserted 1 row with 0 errors.


id,input
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers."


# Making OpenAI API calls

Calling OpenAI API endpoints involves constructing a message object, which we express in Pixeltable by adding a new computed column,i.e., a column that represents a computation.

In [5]:
prompt = "For the following sentence, extract all company names from the text."

msgs = [
    { "role": "system", "content": prompt },
    { "role": "user", "content": t.input }
]

t.add_column(input_msgs=msgs)

Computing cells: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 211.78 cells/s]
Added 1 column value with 0 errors.


UpdateStatus(num_rows=1, num_computed_values=1, num_excs=0, updated_cols=[], cols_with_excs=[])

Unlike the values of the`input` column, which users provide, the `t.input_msgs` column is computed automatically from the `t.input` column values:

In [6]:
t.show()

id,input,input_msgs
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]"


In Pixeltable, OpenAI API calls are exposed as Pixeltable functions, which can be used to create computed columns.

For the following command to work, you must have set `os.environ['OPENAI_API_KEY']`.

In [7]:
from pixeltable.functions import openai

t['chat_output'] = openai.chat_completions(model='gpt-3.5-turbo', messages=t.input_msgs)
t.show()

Computing cells: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25 cells/s]
Added 1 column value with 0 errors.


id,input,input_msgs,chat_output
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-9G7ou4uLNHVoCHNwyLsfhBWVtQiMP', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 69, 'prompt_tokens': 61, 'completion_tokens': 8}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Lehman Brothers, Federal Reserve Bank', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630336, 'system_fingerprint': 'fp_c2295e73ad'}"


The result objects of the OpenAI API calls are generally complex JSON structures, which require some navigation to extract the response. We can express this as JSON path expressions and create another computed column:

In [8]:
t['response'] = t.chat_output.choices[0].message.content
t.show()

Computing cells: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 233.03 cells/s]
Added 1 column value with 0 errors.


id,input,input_msgs,chat_output,response
0,"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","[{'role': 'system', 'content': 'For the following sentence, extract all company names from the text.'}, {'role': 'user', 'content': 'On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.'}]","{'id': 'chatcmpl-9G7ou4uLNHVoCHNwyLsfhBWVtQiMP', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 69, 'prompt_tokens': 61, 'completion_tokens': 8}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Lehman Brothers, Federal Reserve Bank', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630336, 'system_fingerprint': 'fp_c2295e73ad'}","Lehman Brothers, Federal Reserve Bank"


Let's run a query to look only at the input and output:

In [9]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","Lehman Brothers, Federal Reserve Bank"


Once we have defined these computed columns, much like with a spreadsheet, newly inserted `t.input` values trigger computation of all derived columns, such as `t.response`:

In [10]:
t.insert({'id': i, 'input': sample_inputs[i]} for i in range(1, len(sample_inputs)))

Computing cells: 100%|██████████████████████████████████████████| 15/15 [00:02<00:00,  5.73 cells/s]
Inserting rows into `chat_completions_demo`: 5 rows [00:00, 1473.75 rows/s]
Computing cells: 100%|██████████████████████████████████████████| 15/15 [00:02<00:00,  5.71 cells/s]
Inserted 5 rows with 0 errors.


UpdateStatus(num_rows=5, num_computed_values=15, num_excs=0, updated_cols=[], cols_with_excs=[])

In [11]:
t.select(t.input, t.response).show()

input,response
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","Lehman Brothers, Federal Reserve Bank"
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]",1. Lehman\n2. JPMorgan Chase
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.",1. Merrill Lynch\n2. Bank of America
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",- American International Group (AIG)
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]",- The Federal Reserve\n- AIG


# Adding ground truth data

Computed table columns are stored and persisted, avoiding repeated computation.
Pixeltable can be used to conduct experiments, for example to compare prompt variations:

We'll start by creating our ground-truth data manually:

In [12]:
t['ground_truth'] = pxt.StringType(nullable=True)

ground_truth = [
    'Lehman Brothers',
    'Lehman Brothers',
    'JP Morgan Chase, Lehman Brothers',
    'Merill Lynch, Bank of America',
    'American International Group',
    'American International Group',
]

for i, gt in enumerate(ground_truth):
    t.update({'ground_truth': gt}, where=(t.id == i))

Added 6 column values with 0 errors.
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 584.16 rows/s]
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 919.20 rows/s]
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 868.93 rows/s]
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 557.75 rows/s]
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 937.07 rows/s]
Inserting rows into `chat_completions_demo`: 1 rows [00:00, 929.38 rows/s]


And this is what we have so far:

In [13]:
t.select(t.input, t.response, t.ground_truth).show()

input,response,ground_truth
"On Sunday, September 14, it was announced that Lehman Brothers would file for bankruptcy after the Federal Reserve Bank declined to participate in creating a financial support facility for Lehman Brothers.","Lehman Brothers, Federal Reserve Bank",Lehman Brothers
The significance of the Lehman Brothers bankruptcy is disputed with some assigning it a pivotal role in the unfolding of subsequent events.,Lehman Brothers,Lehman Brothers
"The principals involved, Ben Bernanke and Henry Paulson, dispute this view, citing a volume of toxic assets at Lehman which made a rescue impossible.[16][17] Immediately following the bankruptcy, JPMorgan Chase provided the broker dealer unit of Lehman Brothers with $138 billion to ""settle securities transactions with customers of Lehman and its clearance parties"" according to a statement made in a New York City Bankruptcy court filing.[18]",1. Lehman\n2. JPMorgan Chase,"JP Morgan Chase, Lehman Brothers"
"The same day, the sale of Merrill Lynch to Bank of America was announced.[19] The beginning of the week was marked by extreme instability in global stock markets, with dramatic drops in market values on Monday, September 15, and Wednesday, September 17.",1. Merrill Lynch\n2. Bank of America,"Merill Lynch, Bank of America"
"The Federal Reserve, at AIG's request, and after AIG had shown that it could not find lenders willing to save it from insolvency, created a credit facility for up to US$85 billion in exchange for a 79.9% equity interest, and the right to suspend dividends to previously issued common and preferred stock.[20]",- The Federal Reserve\n- AIG,American International Group
"On September 16, the large insurer American International Group (AIG), a significant participant in the credit default swaps markets, suffered a liquidity crisis following the downgrade of its credit rating.",- American International Group (AIG),American International Group


# Evaluation

Now that we have some ground truth available, we can carry out basic evaluations of the GPT outputs, in this case by asking ChatGPT to decide whether the two are equivalent. 

To start with, we'll create an evaluation prompt:

In [14]:
system_prompt = '''
Compare the following listA and listB of entities, and check if they contain the same entities.
Return a json object with the following format:
{"reasoning": explaining your reasoning, "decision": 1 if the lists matched, 0 otherwise}
'''

@pxt.udf
def user_prompt(listA: str, listB: str) -> str:
    return f'listA: "{listA}" \n listB: "{listB}"'

t.add_column(
    eval_prompt=[
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": user_prompt(t.response, t.ground_truth) }
    ])

Computing cells: 100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 549.45 cells/s]
Added 6 column values with 0 errors.


UpdateStatus(num_rows=6, num_computed_values=6, num_excs=0, updated_cols=[], cols_with_excs=[])

In [15]:
t.select(t.eval_prompt).show()

eval_prompt
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""1. Merrill Lynch 2. Bank of America"" listB: ""Merill Lynch, Bank of America""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""- The Federal Reserve - AIG"" listB: ""American International Group""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""Lehman Brothers, Federal Reserve Bank"" listB: ""Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""1. Lehman 2. JPMorgan Chase"" listB: ""JP Morgan Chase, Lehman Brothers""'}]"
"[{'role': 'system', 'content': ' Compare the following listA and listB of entities, and check if they contain the same entities. Return a json object with the following format: {""reasoning"": explaining your reasoning, ""decision"": 1 if the lists matched, 0 otherwise} '}, {'role': 'user', 'content': 'listA: ""- American International Group (AIG)"" listB: ""American International Group""'}]"


The actual evaluation happens in another computed column. We can use OpenAI's handy `response_format` parameter to enforce that the output is properly formed JSON.

In [16]:
t['eval'] = openai.chat_completions(model='gpt-3.5-turbo', messages=t.eval_prompt, response_format={'type': 'json_object'})
t['eval_output'] = t.eval.choices[0].message.content

Computing cells: 100%|████████████████████████████████████████████| 6/6 [00:06<00:00,  1.05s/ cells]
Added 6 column values with 0 errors.
Computing cells: 100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 679.57 cells/s]
Added 6 column values with 0 errors.


Let's take a look:

In [17]:
t.select(t.eval, t.eval_output).show()

eval,eval_output
"{'id': 'chatcmpl-9G7pLpT0mfYMSG5XpElAfCNme1Vg1', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 113, 'prompt_tokens': 82, 'completion_tokens': 31}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The entities in listA and listB are the same (both \'Lehman Brothers\')."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630363, 'system_fingerprint': 'fp_c2295e73ad'}","{""reasoning"": ""The entities in listA and listB are the same (both 'Lehman Brothers')."", ""decision"": 1}"
"{'id': 'chatcmpl-9G7pNHOzwtt3cCYsc82jbMQuXGmDe', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 146, 'prompt_tokens': 84, 'completion_tokens': 62}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n ""reasoning"": ""ListA contains entities \'The Federal Reserve\' and \'AIG\', while ListB contains the entity \'American International Group\'. \'AIG\' in ListA most likely refers to \'American International Group\'. Hence, the lists match."",\n ""decision"": 1\n}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630365, 'system_fingerprint': 'fp_c2295e73ad'}","{\n ""reasoning"": ""ListA contains entities 'The Federal Reserve' and 'AIG', while ListB contains the entity 'American International Group'. 'AIG' in ListA most likely refers to 'American International Group'. Hence, the lists match."",\n ""decision"": 1\n}"
"{'id': 'chatcmpl-9G7pPNFYpBZMkqj8C8uPDqduQIXek', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 165, 'prompt_tokens': 92, 'completion_tokens': 73}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{""reasoning"": ""The entities in listA and listB are the same, but they are formatted differently. \'JP Morgan Chase\' in listB is the same as \'JPMorgan Chase\' in listA. \'Lehman Brothers\' in listB is the same as \'Lehman\' in listA."", ""decision"": 1}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630367, 'system_fingerprint': 'fp_c2295e73ad'}","{""reasoning"": ""The entities in listA and listB are the same, but they are formatted differently. 'JP Morgan Chase' in listB is the same as 'JPMorgan Chase' in listA. 'Lehman Brothers' in listB is the same as 'Lehman' in listA."", ""decision"": 1}"
"{'id': 'chatcmpl-9G7pMqB7iH4FwOsxhUIoJLSszDVWr', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 149, 'prompt_tokens': 91, 'completion_tokens': 58}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n ""reasoning"": ""ListA has entities with numbers and periods, while listB does not. The spelling of \'Merrill Lynch\' is also different in listB. Therefore, the entities in the two lists do not match."",\n ""decision"": 0\n}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630364, 'system_fingerprint': 'fp_c2295e73ad'}","{\n ""reasoning"": ""ListA has entities with numbers and periods, while listB does not. The spelling of 'Merrill Lynch' is also different in listB. Therefore, the entities in the two lists do not match."",\n ""decision"": 0\n}"
"{'id': 'chatcmpl-9G7pOAcuWtKuInEcv5rQhXwp5a71Q', 'model': 'gpt-3.5-turbo-0125', 'usage': {'total_tokens': 120, 'prompt_tokens': 86, 'completion_tokens': 34}, 'object': 'chat.completion', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n ""reasoning"": ""ListA contains an additional entity (Federal Reserve Bank) that is not present in ListB."",\n ""decision"": 0\n}', 'tool_calls': None, 'function_call': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'created': 1713630366, 'system_fingerprint': 'fp_c2295e73ad'}","{\n ""reasoning"": ""ListA contains an additional entity (Federal Re

Finally, it's time to pull the `decision` out of the JSON structs returned by OpenAI. There's just one complication: the `chat_completions` responses are strings, not JSON structs. We can resolve this with Pixeltable's handy `apply` method, which turns any Python function into a Pixeltable function. In this case, we'll apply the Python function `json.loads` to parse the string into a JSON struct.

In [24]:
import json

t['eval_decision'] = t.eval_output.apply(json.loads).decision

Computing cells: 100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 661.61 cells/s]
Added 6 column values with 0 errors.


In [25]:
t.select(t.eval_output, t.eval_decision).show()

eval_output,eval_decision
"{""reasoning"": ""The entities in listA and listB are the same (both 'Lehman Brothers')."", ""decision"": 1}",1
"{\n ""reasoning"": ""ListA contains an additional entity (Federal Reserve Bank) that is not present in ListB."",\n ""decision"": 0\n}",0
"{\n ""reasoning"": ""ListA contains entities 'The Federal Reserve' and 'AIG', while ListB contains the entity 'American International Group'. 'AIG' in ListA most likely refers to 'American International Group'. Hence, the lists match."",\n ""decision"": 1\n}",1
"{""reasoning"": ""The entities in listA and listB are the same, but they are formatted differently. 'JP Morgan Chase' in listB is the same as 'JPMorgan Chase' in listA. 'Lehman Brothers' in listB is the same as 'Lehman' in listA."", ""decision"": 1}",1
"{\n ""reasoning"": ""ListA has entities with numbers and periods, while listB does not. The spelling of 'Merrill Lynch' is also different in listB. Therefore, the entities in the two lists do not match."",\n ""decision"": 0\n}",0
"{""reasoning"": ""The entities in listA and listB refer to the same company 'American International Group', the only difference is the abbreviation 'AIG' in listA."", ""decision"": 1}",1
